## Increment the database Size

### Starting methods

In [8]:
import mne.io

import numpy as np
import pandas as pd
import os

In [15]:
def csv_to_raw_array(csv_file_path, sfreq=250):
    """
    Convert a CSV file with EEG channel data into an mne.io.RawArray object.
    
    Parameters:
    csv_file_path (str): Path to the CSV file.
    sfreq (float): Sampling frequency of the data. Default is 250 Hz.
    
    Returns:
    raw (mne.io.RawArray): The raw EEG data as an MNE RawArray object.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Split the space-separated values and convert to floats
    data = []
    for column in df.columns:
        # For each column, split the string values into floats
        channel_data = [list(map(float, value.split())) for value in df[column]]
        # Flatten the list of lists
        channel_data_flat = [item for sublist in channel_data for item in sublist]
        data.append(channel_data_flat)

    # Convert the data to a NumPy array and transpose it
    data = np.array(data)
    
    # Define the channel names (from your original header)
    channel_names = ['F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 
                     'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']
    
    # Define the channel types (assuming EEG for all)
    channel_types = ['eeg'] * len(channel_names)
    
    # Create the info object
    info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types=channel_types)
    
    # Create the RawArray object
    raw = mne.io.RawArray(data, info)
    raw.pick('eeg')
    raw.set_eeg_reference('average')
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(ten_twenty_montage)
    
    return raw

for filename in os.listdir(r'D:\Codes\NEWTCC\database\csv\norm'):
    if filename.endswith(".csv"):
        csv_file_path = os.path.join(r'D:\Codes\NEWTCC\database\csv\norm', filename)
        raw = csv_to_raw_array(csv_file_path)
        


Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    30.716 secs
Ready.
<RawArray | 16 x 7680 (30.7 s), ~977 kB, data loaded>
Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    30.716 secs
Ready.
<RawArray | 16 x 7680 (30.7 s), ~977 kB, data loaded>
Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    30.716 secs
Ready.
<RawArray | 16 x 7680 (30.7 s), ~977 kB, data loaded>
Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    30.716 secs
Ready.
<RawArray | 16 x 7680 (30.7 s), ~977 kB, data loaded>
Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    30.716 secs
Ready.
<RawArray | 16 x 7680 (30.7 s), ~977 kB, data loaded>
Creating RawArray with float64 data, n_channels=16, n_times=7680
    Range : 0 ... 7679 =      

In [6]:
import os
import csv

def eea_to_channel_csv(eea_file_path, csv_file_path, rows_per_channel=7680):
    """
    Convert an .eea file to a .csv format, where every 7680 rows are grouped
    under a single CSV row with specific EEG channel headers.
    
    Parameters:
    eea_file_path (str): Path to the input .eea file.
    csv_file_path (str): Path to the output .csv file.
    rows_per_channel (int): Number of rows that correspond to each channel's data. Default is 7680.
    """
    # Define the channel headers in the specified order
    channel_headers = ['F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 
                       'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']

    try:
        # Open the EEA file (assuming it's text-based for this example)
        with open(eea_file_path, 'r') as eea_file:
            lines = eea_file.readlines()

        # Open the CSV file for writing
        with open(csv_file_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file)

            # Write the header row
            writer.writerow(channel_headers)

            # Loop through the file and write rows based on the number of channels
            for i in range(0, len(lines), rows_per_channel * len(channel_headers)):
                # Collect data for each channel in order
                row = []
                for channel_index in range(len(channel_headers)):
                    # Extract the block of data for the current channel
                    channel_data = lines[i + channel_index * rows_per_channel : i + (channel_index + 1) * rows_per_channel]
                    
                    # Flatten the channel data into a single row
                    row.append(' '.join([line.strip() for line in channel_data]))

                # Write the row to the CSV
                writer.writerow(row)

        print(f"Conversion successful! CSV saved at: {csv_file_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

def convert_all_eea_in_directory(directory_path, output_directory):
    """
    Convert all .eea files in the given directory to .csv files and save them in the output directory.
    
    Parameters:
    directory_path (str): Path to the directory containing .eea files.
    output_directory (str): Path to the directory where .csv files will be saved.
    """
    # Ensure output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Loop over all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".eea"):
            eea_file_path = os.path.join(directory_path, filename)
            csv_file_path = os.path.join(output_directory, filename.replace('.eea', '.csv'))
            print(f"Converting {filename} to CSV...")
            eea_to_channel_csv(eea_file_path, csv_file_path)

# Example usage:
convert_all_eea_in_directory(r'D:\Codes\NEWTCC\database\sch', r'D:\Codes\NEWTCC\database\csv\schz')


Converting 022w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\022w1.csv
Converting 088w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\088w1.csv
Converting 103w.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\103w.csv
Converting 113w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\113w1.csv
Converting 155w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\155w1.csv
Converting 156w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\156w1.csv
Converting 192w.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\192w.csv
Converting 219w1.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\219w1.csv
Converting 221w.eea to CSV...
Conversion successful! CSV saved at: D:\Codes\NEWTCC\database\csv\schz\221w.cs